In [ ]:
!git clone https://github.com/ipostoev/SEMarkup-2023.git
!mv /content/SEMarkup-2023 /content/semarkup
!cd semarkup && git checkout develop
!pip install -r "/content/semarkup/baseline/requirements.txt"

In [ ]:
import sys
sys.path.append("/content/semarkup/baseline/parser/")

import torch
import dataset_reader
from feedforward_classifier import FeedForwardClassifier
from dependency_classifier import DependencyClassifier
from semparser import MorphoSyntaxSemanticParser

from allennlp.data import Instance, Token, Vocabulary
from allennlp.data.data_loaders import MultiProcessDataLoader
from allennlp.data.token_indexers.pretrained_transformer_mismatched_indexer import PretrainedTransformerMismatchedIndexer
from allennlp.modules.token_embedders.pretrained_transformer_mismatched_embedder import PretrainedTransformerMismatchedEmbedder
from allennlp.nn.util import get_text_field_mask

In [ ]:
reader = dataset_reader.ComprenoUDDatasetReader({'tokens': PretrainedTransformerMismatchedIndexer('cointegrated/rubert-tiny')})
instances = list(reader.read("/content/drive/MyDrive/semarkup/val_splitted.conllu"))
vocab = Vocabulary.from_instances(instances)
embedder = PretrainedTransformerMismatchedEmbedder(model_name="cointegrated/rubert-tiny")

In [ ]:
emb_output_dim = embedder.get_output_dim()
lemma_rule_classifier = FeedForwardClassifier(vocab, emb_output_dim, 256, vocab.get_vocab_size("lemma_rule_labels"), "relu", 0.1)
pos_feats_classifier = FeedForwardClassifier(vocab, emb_output_dim, 256, vocab.get_vocab_size("pos_feats_labels"), "relu", 0.1)
depencency_classifier = DependencyClassifier(vocab, emb_output_dim, 512, vocab.get_vocab_size("deprel_labels"), "relu", 0.1)
semslot_classifier = FeedForwardClassifier(vocab, emb_output_dim, 1024, vocab.get_vocab_size("semslot_labels"), "relu", 0.1)
semclass_classifier = FeedForwardClassifier(vocab, emb_output_dim, 1024, vocab.get_vocab_size("semclass_labels"), "relu", 0.1)
model = MorphoSyntaxSemanticParser(vocab, embedder, lemma_rule_classifier, pos_feats_classifier, depencency_classifier, semslot_classifier, semclass_classifier)